# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. Elkin Castillo
2. Camilo Figueroa
3. Joan Sebastián Jimenez
4. Lina María Moreno Vásquez


In [15]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [16]:
#ContextoSpark.stop() #para detener el contexto
#ContextoSql.sql('DROP TABLE asistencia_sql') # para eliminar una tabla de sql

In [17]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...
!hdfs dfs -copyFromLocal Asistencia_del_Pitch.csv /tmp/
!hdfs dfs -copyFromLocal Evaluacion_del_Pitch.csv /tmp/
!hdfs dfs -ls /tmp/*csv

copyFromLocal: `/tmp/Asistencia_del_Pitch.csv': File exists
copyFromLocal: `/tmp/Evaluacion_del_Pitch.csv': File exists
-rw-r--r--   1 root supergroup      93121 2021-05-07 01:20 /tmp/Asistencia_del_Pitch.csv
-rw-r--r--   1 root supergroup     111944 2021-05-07 01:20 /tmp/Evaluacion_del_Pitch.csv


In [18]:
!hdfs dfs -cat /tmp/Asistencia_del_Pitch.csv | head -n3

"Marca temporal","Nombre de usuario","Equipo al que perteneces:","Equipo que va a exponer:"
"2021/04/08 8:07:55 a.��m. GMT-5","jppinedal@unal.edu.co","Pertenezco a: Servicios a domicilio","Expone: Servicios a domicilio"
"2021/04/08 8:09:39 a.��m. GMT-5","dgarciabl@unal.edu.co","Pertenezco a: Lo tengo y te lo presto.","Expone: Lo tengo y te lo presto."
cat: Unable to write to output stream.


In [19]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...
asistencia = SesionSpark.read.load('/tmp/Asistencia_del_Pitch.csv', format = 'csv', sep = ',', inferSchema = 'true', header= 'true')
asistencia.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo al que perteneces:: string (nullable = true)
 |-- Equipo que va a exponer:: string (nullable = true)



In [20]:
evaluacion = SesionSpark.read.load('/tmp/Evaluacion_del_Pitch.csv', format = 'csv', sep = ',', inferSchema = 'true', header= 'true')
evaluacion.printSchema()

root
 |-- Marca temporal: string (nullable = true)
 |-- Nombre de usuario: string (nullable = true)
 |-- Equipo que vas a evaluar:: string (nullable = true)
 |-- Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?: integer (nullable = true)
 |-- Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?: integer (nullable = true)
 |-- Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?: integer (nullable = true)
 |-- Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?: integer (nullable = true)
 |-- Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?: integer (nullable = true)
 |-- Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.: integer (nullable = true)
 |-- Tracción: El equipo responde adecuadamente si cuenta con clientes que demues

In [21]:
#cambiamos los nombres de las columnas:
from pyspark.sql.functions import col

In [22]:
asistencia = asistencia.select(col("Marca temporal").alias("fecha"), col("Nombre de usuario").alias("correo"), col("Equipo al que perteneces:").alias("equipo"), col("Equipo que va a exponer:").alias("equipo_expositor"))
asistencia.show(10)


+--------------------+--------------------+--------------------+--------------------+
|               fecha|              correo|              equipo|    equipo_expositor|
+--------------------+--------------------+--------------------+--------------------+
|2021/04/08 8:07:5...|jppinedal@unal.ed...|Pertenezco a: Ser...|Expone: Servicios...|
|2021/04/08 8:09:3...|dgarciabl@unal.ed...|Pertenezco a: Lo ...|Expone: Lo tengo ...|
|2021/04/08 8:09:5...|auarbelaeza@unal....|Pertenezco a: Ges...|Expone: Te lo cam...|
|2021/04/08 8:09:5...|jmunozhe@unal.edu.co|Pertenezco a: Lo ...|Expone: Te lo cam...|
|2021/04/08 8:10:0...|dgerenal@unal.edu.co|Pertenezco a: Te ...|Expone: Te lo cam...|
|2021/04/08 8:10:1...|nvalenciat@unal.e...|Pertenezco a: Te ...|Expone: Te lo cam...|
|2021/04/08 8:10:1...|fmiranda@unal.edu.co|Pertenezco a: Mi ...|Expone: Te lo cam...|
|2021/04/08 8:10:2...|jucardonaa@unal.e...|Pertenezco a: Ges...|Expone: Te lo cam...|
|2021/04/08 8:10:2...|seagudeloo@unal.e...|Pertenezco 

In [23]:
evaluacion = evaluacion.select(
	col("Marca temporal").alias("fecha"),
	col("Nombre de usuario").alias("correo"),
	col("Equipo que vas a evaluar:").alias("equipo_expositor"),
	col("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?").alias("p1"),
	col("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?").alias("p2"),
	col("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?").alias("p3"),
	col("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?").alias("p4"),
	col("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?").alias("p5"),
	col("`Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.`").alias("p6"),
	col("`Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.`").alias("p7"),
	col("`Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.`").alias("p8"),
	col("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?").alias("p9"),
	col("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ").alias("p10"),
	col("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?").alias("p11"),
	col("`Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.`").alias("p12"),
	col("Exposición: ¿Qué tan coordinados estaban los expositores?").alias("p13"),
	col("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?").alias("p14"),
	col("`Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.`").alias("p15"),
	col("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)").alias("invertiria"),
	col("`Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.`").alias("observaciones")
	)

evaluacion.show(5)

+--------------------+--------------------+----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+--------------------+
|               fecha|              correo|equipo_expositor| p1| p2| p3| p4| p5| p6| p7| p8| p9|p10|p11|p12|p13|p14|p15|invertiria|       observaciones|
+--------------------+--------------------+----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+--------------------+
|2021/04/08 8:26:3...|jmunozbo@unal.edu.co|   Te lo cambio.|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  3|  5|  5|  5|  5|   Tal vez|                null|
|2021/04/08 8:26:4...|daestradam@unal.e...|   Te lo cambio.|  5|  5|  5|  4|  5|  5|  4|  5|  4|  4|  4|  4|  4|  5|  5|   Tal vez|                null|
|2021/04/08 8:27:2...|seagudeloo@unal.e...|   Te lo cambio.|  4|  4|  4|  4|  4|  4|  4|  4|  3|  4|  3|  5|  5|  4|  5|        Sí|                null|
|2021/04/08 8:27:3...|jubuitrago@unal.e...|   Te lo cambio.|  5|  4|  5|  4|  5|  

In [24]:
## Agregamos de una vez la calificación total de cada estudiante como un promedio de las 15 preguntas.
from pyspark.sql.functions import col

columnas = [col('p1'), col('p2'), col('p3'), col('p4'), col('p5'), col('p6'), col('p7'), col('p8'), col('p9'), col('p10'), col('p11'), col('p12'), col('p13'), col('p14'), col('p15')]
funcion_promedio = sum(x for x in columnas)/len(columnas)
evaluacion = evaluacion.withColumn('promedio', funcion_promedio)
evaluacion.show(5)

+--------------------+--------------------+----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+--------------------+------------------+
|               fecha|              correo|equipo_expositor| p1| p2| p3| p4| p5| p6| p7| p8| p9|p10|p11|p12|p13|p14|p15|invertiria|       observaciones|          promedio|
+--------------------+--------------------+----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----------+--------------------+------------------+
|2021/04/08 8:26:3...|jmunozbo@unal.edu.co|   Te lo cambio.|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  3|  5|  5|  5|  5|   Tal vez|                null| 4.866666666666666|
|2021/04/08 8:26:4...|daestradam@unal.e...|   Te lo cambio.|  5|  5|  5|  4|  5|  5|  4|  5|  4|  4|  4|  4|  4|  5|  5|   Tal vez|                null| 4.533333333333333|
|2021/04/08 8:27:2...|seagudeloo@unal.e...|   Te lo cambio.|  4|  4|  4|  4|  4|  4|  4|  4|  3|  4|  3|  5|  5|  4|  5|        Sí|         

In [26]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

asistencia.createOrReplaceTempView('vista_asistencia')    

In [27]:
ContextoSql.sql('show tables').show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|        |vista_asistencia|       true|
+--------+----------------+-----------+



In [28]:
listado_estudiantes = ContextoSql.sql("""
    SELECT correo 
    FROM vista_asistencia 
    GROUP By correo
    ORDER BY correo 
    """).show(10)

+--------------------+
|              correo|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
+--------------------+
only showing top 10 rows



In [29]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...
cant_asistencias =  ContextoSql.sql("""
    SELECT correo, count(*) AS asistencias, MAX(fecha), MIN(fecha)
    FROM vista_asistencia
    GROUP BY correo
    """)

cant_asistencias.show(5) 

+--------------------+-----------+--------------------+--------------------+
|              correo|asistencias|          max(fecha)|          min(fecha)|
+--------------------+-----------+--------------------+--------------------+
|samolinap@unal.ed...|         10|2021/04/13 9:37:2...|2021/04/08 8:11:4...|
| jruedat@unal.edu.co|          8|2021/04/13 9:41:2...|2021/04/08 8:12:5...|
|nvalenciat@unal.e...|          8|2021/04/13 9:39:3...|2021/04/08 8:10:1...|
|serendona@unal.ed...|          8|2021/04/13 9:40:2...|2021/04/08 8:10:4...|
|jgutierrezlo@unal...|          8|2021/04/13 9:39:0...|2021/04/08 8:12:0...|
+--------------------+-----------+--------------------+--------------------+
only showing top 5 rows



In [30]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...
pocas_presentaciones = ContextoSql.sql("""
    SELECT correo, count(*)  
    FROM vista_asistencia   
    GROUP BY correo 
    having count(*) <=2 
    """)
pocas_presentaciones.show()

+--------------------+--------+
|              correo|count(1)|
+--------------------+--------+
|jdroldano@unal.ed...|       1|
+--------------------+--------+



In [ ]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...    
# R/ De la respuesta del punto 4 observamos que solo hay 1 estudiante que asistió a una sola presentación. Ahora, para determinar 
# quién no asistió a niguna presentación, necesitaríamos el listado de alumnos matriculados en el curso, de esta forma podríamos
# hacer el cruce entre ambas tablas y verificar quién está inscrito en el curso y no está en la lista de asistencia.

In [31]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...
integrantes = ContextoSql.sql("""
    SELECT substring(equipo, 15) AS equipo, correo
    FROM vista_asistencia
    GROUP BY equipo, correo
    ORDER BY equipo, correo
    """)
integrantes.show(20)

+--------------------+--------------------+
|              equipo|              correo|
+--------------------+--------------------+
|Administración de...|angutierrezb@unal...|
|Administración de...| eapenad@unal.edu.co|
|Administración de...|emflorezb@unal.ed...|
|Administración de...|framirezf@unal.ed...|
|Administración de...|jhcordoba@unal.ed...|
|Administración de...| jortize@unal.edu.co|
|Administración de...| kmolano@unal.edu.co|
|Administración de...|lumarinb@unal.edu.co|
|Administración de...|nsalinas@unal.edu.co|
|Administración de...|rmbuilesm@unal.ed...|
|Administración de...|saespinosab@unal....|
|Administración de...|samolinap@unal.ed...|
|Administración de...|sednarvaezna@unal...|
|Gestión de Menús ...| abanolc@unal.edu.co|
|Gestión de Menús ...|anapariciom@unal....|
|Gestión de Menús ...|davgarciava@unal....|
|Gestión de Menús ...|emlopezr@unal.edu.co|
|Gestión de Menús ...|favasquezj@unal.e...|
|Gestión de Menús ...|hriveraa@unal.edu.co|
|Gestión de Menús ...|jcardonaso

In [32]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...
xasistencia =  ContextoSql.sql("""
    SELECT fecha, correo, substring(equipo, 15) AS equipo, substring(equipo_expositor, 9) AS equipo_expositor
    FROM vista_asistencia
    """)
xasistencia.show(10)   

+--------------------+--------------------+--------------------+--------------------+
|               fecha|              correo|              equipo|    equipo_expositor|
+--------------------+--------------------+--------------------+--------------------+
|2021/04/08 8:07:5...|jppinedal@unal.ed...|Servicios a domic...|Servicios a domic...|
|2021/04/08 8:09:3...|dgarciabl@unal.ed...|Lo tengo y te lo ...|Lo tengo y te lo ...|
|2021/04/08 8:09:5...|auarbelaeza@unal....|Gestión de Reside...|       Te lo cambio.|
|2021/04/08 8:09:5...|jmunozhe@unal.edu.co|Lo tengo y te lo ...|       Te lo cambio.|
|2021/04/08 8:10:0...|dgerenal@unal.edu.co|       Te lo cambio.|       Te lo cambio.|
|2021/04/08 8:10:1...|nvalenciat@unal.e...|       Te lo cambio.|       Te lo cambio.|
|2021/04/08 8:10:1...|fmiranda@unal.edu.co|     Mi Sanandresito|       Te lo cambio.|
|2021/04/08 8:10:2...|jucardonaa@unal.e...|Gestión de progra...|       Te lo cambio.|
|2021/04/08 8:10:2...|seagudeloo@unal.e...|Gestión de 

In [33]:
xasistencia.createOrReplaceTempView('vista_xasistencia')
ContextoSql.sql('show tables').show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|        | vista_asistencia|       true|
|        |vista_xasistencia|       true|
+--------+-----------------+-----------+



In [34]:
asist_x_present = ContextoSql.sql("""
                        SELECT equipo_expositor, count(*) As asistentes_presentacion
                        FROM vista_xasistencia
                        WHERE equipo <> equipo_expositor 
                        GROUP BY equipo_expositor
                        """)
asist_x_present.show()    

+--------------------+-----------------------+
|    equipo_expositor|asistentes_presentacion|
+--------------------+-----------------------+
|Servicios a domic...|                     76|
|Lo tengo y te lo ...|                     75|
|Gestión de Reside...|                     72|
|       Te lo cambio.|                     75|
|Administración de...|                     70|
|Deserción de estu...|                      3|
|Gestión de Menús ...|                     72|
|Gestión de progra...|                     74|
|     Mi Sanandresito|                     72|
+--------------------+-----------------------+



In [35]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...
evaluacion.createOrReplaceTempView('vista_evaluacion')
ContextoSql.sql('show tables').show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|        | vista_asistencia|       true|
|        | vista_evaluacion|       true|
|        |vista_xasistencia|       true|
+--------+-----------------+-----------+



In [36]:
## las siguientes son las personas que evaluaron a su propio equipo
y = ContextoSql.sql("""
    SELECT DISTINCT(correo)  
    FROM vista_evaluacion
    LEFT JOIN vista_xasistencia USING (correo)
    WHERE equipo = vista_evaluacion.equipo_expositor
    """)
y.show(20)

+--------------------+
|              correo|
+--------------------+
| jruedat@unal.edu.co|
| kmolano@unal.edu.co|
|mcarvajalsa@unal....|
|daestradam@unal.e...|
|jhcordoba@unal.ed...|
|  lmayar@unal.edu.co|
|hriveraa@unal.edu.co|
|anapariciom@unal....|
+--------------------+



In [37]:
## 8.1 A continuación se observa que hubo estudiantes en el archivo de Asistencia que firmaron como si pertenecieran
## a dos equipos, ejemplo: anapariciom. En  total son 4 estudiantes donde pasa esto. Se considera esto como 
# un error al momento de registrar la asistencia y se dejó al estudiante como si perteneciera al equipo 
# que más asistencias registró el estudiante (en el caso de anapariciom queda asignada a Gestión de Menús):

pers_equ = ContextoSql.sql("""
    SELECT  correo, equipo, COUNT(*) AS conteo
    FROM vista_xasistencia
    GROUP BY correo, equipo
    ORDER BY correo
    """)
pers_equ.show(5)

+--------------------+--------------------+------+
|              correo|              equipo|conteo|
+--------------------+--------------------+------+
| abanolc@unal.edu.co|Gestión de Menús ...|     7|
|acastrillonv@unal...|Gestión de progra...|     8|
| aflemag@unal.edu.co|     Mi Sanandresito|     8|
|anapariciom@unal....|Servicios a domic...|     1|
|anapariciom@unal....|Gestión de Menús ...|     9|
+--------------------+--------------------+------+
only showing top 5 rows



In [38]:
pers_equ.createOrReplaceTempView('vista_pers_equ')
ContextoSql.sql('show tables').show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|        | vista_asistencia|       true|
|        | vista_evaluacion|       true|
|        |   vista_pers_equ|       true|
|        |vista_xasistencia|       true|
+--------+-----------------+-----------+



In [39]:
## Se identifican los repetidos (los 4 estudiantes que firmaron como si pertenecieran a dos equipos.
## las filas quedan como num_fila = 2)
pers_equ2 = ContextoSql.sql("""
    SELECT *, ROW_NUMBER ( ) OVER ( PARTITION BY correo ORDER BY conteo DESC) AS num_fila
    FROM vista_pers_equ
    ORDER BY num_fila DESC
    """)
pers_equ2.show(10)

+--------------------+--------------------+------+--------+
|              correo|              equipo|conteo|num_fila|
+--------------------+--------------------+------+--------+
| kmolano@unal.edu.co|Administración de...|     1|       2|
|anapariciom@unal....|Servicios a domic...|     1|       2|
|jhcordoba@unal.ed...|Administración de...|     1|       2|
|mcarvajalsa@unal....|Gestión de Reside...|     1|       2|
| aflemag@unal.edu.co|     Mi Sanandresito|     8|       1|
|cjfunezg@unal.edu.co|Servicios a domic...|     8|       1|
|angutierrezb@unal...|Administración de...|     8|       1|
|anoriega@unal.edu.co|       Te lo cambio.|     8|       1|
|auarbelaeza@unal....|Gestión de Reside...|     8|       1|
|anapariciom@unal....|Gestión de Menús ...|     9|       1|
+--------------------+--------------------+------+--------+
only showing top 10 rows



In [40]:
pers_equ2.createOrReplaceTempView('vista_pers_equ2')
ContextoSql.sql('show tables').show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|        | vista_asistencia|       true|
|        | vista_evaluacion|       true|
|        |   vista_pers_equ|       true|
|        |  vista_pers_equ2|       true|
|        |vista_xasistencia|       true|
+--------+-----------------+-----------+



In [41]:
## se trabaja solo con los num_fila = 1.
pers_equ3 = ContextoSql.sql("""
    SELECT *
    FROM vista_pers_equ2
    WHERE num_fila = 1
    """)
pers_equ3.show(10)    

+--------------------+--------------------+------+--------+
|              correo|              equipo|conteo|num_fila|
+--------------------+--------------------+------+--------+
|auarbelaeza@unal....|Gestión de Reside...|     8|       1|
|angutierrezb@unal...|Administración de...|     8|       1|
| aflemag@unal.edu.co|     Mi Sanandresito|     8|       1|
| abanolc@unal.edu.co|Gestión de Menús ...|     7|       1|
|anapariciom@unal....|Gestión de Menús ...|     9|       1|
|anoriega@unal.edu.co|       Te lo cambio.|     8|       1|
|daestradam@unal.e...|Lo tengo y te lo ...|     8|       1|
| dadazam@unal.edu.co|     Mi Sanandresito|     9|       1|
|cquinchiar@unal.e...|Gestión de Reside...|     6|       1|
| bocampo@unal.edu.co|Servicios a domic...|     8|       1|
+--------------------+--------------------+------+--------+
only showing top 10 rows



In [42]:
pers_equ3.createOrReplaceTempView('vista_pers_equ3')
ContextoSql.sql('show tables').show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|        | vista_asistencia|       true|
|        | vista_evaluacion|       true|
|        |   vista_pers_equ|       true|
|        |  vista_pers_equ2|       true|
|        |  vista_pers_equ3|       true|
|        |vista_xasistencia|       true|
+--------+-----------------+-----------+



In [43]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

nota_prom = ContextoSql.sql("""
    SELECT ROUND(AVG(p1),1) AS avg1, ROUND(AVG(p2),1) AS avg2, ROUND(AVG(p3),1) AS avg3, ROUND(AVG(p4),1) AS avg4, 
            ROUND(AVG(p5),1) AS avg5, ROUND(AVG(p6),1) AS avg6, ROUND(AVG(p7),1) AS avg7, ROUND(AVG(p8),1) AS avg8,
            ROUND(AVG(p9),1) AS avg9, ROUND(AVG(p10),1) AS avg10, ROUND(AVG(p11),1) AS avg11, ROUND(AVG(p12),1) AS avg12, 
            ROUND(AVG(p13),1) AS avg13, ROUND(AVG(p14),1) AS avg14, ROUND(AVG(p15),1) AS avg15
    FROM vista_evaluacion
    LEFT JOIN vista_pers_equ3 USING (correo)
    WHERE equipo <> vista_evaluacion.equipo_expositor
    """)
nota_prom.show(30)


+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+
|avg1|avg2|avg3|avg4|avg5|avg6|avg7|avg8|avg9|avg10|avg11|avg12|avg13|avg14|avg15|
+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+
| 4.4| 4.3| 4.4| 4.2| 4.4| 4.4| 4.2| 4.3| 4.2|  4.4|  4.2|  4.5|  4.4|  4.4|  4.5|
+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+



In [44]:
## Promedio total de cada equipo:
nota_prom_equ = ContextoSql.sql("""
    SELECT vista_evaluacion.equipo_expositor, ROUND(AVG(promedio),1) AS promedio_tot
    FROM vista_evaluacion
    LEFT JOIN vista_pers_equ3 USING (correo)
    WHERE equipo <> vista_evaluacion.equipo_expositor
    GROUP BY vista_evaluacion.equipo_expositor
    """)
nota_prom_equ.show(30)

+--------------------+------------+
|    equipo_expositor|promedio_tot|
+--------------------+------------+
|Servicios a domic...|         4.4|
|Lo tengo y te lo ...|         4.5|
|Gestión de Reside...|         4.5|
|       Te lo cambio.|         4.3|
|Administración de...|         4.2|
|Deserción de estu...|         4.8|
|Gestión de Menús ...|         4.4|
|     Mi Sanandresito|         4.2|
|Gestión de progra...|         4.4|
+--------------------+------------+



In [45]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...
nota_item_equ = ContextoSql.sql("""
    SELECT vista_evaluacion.equipo_expositor, ROUND(AVG(p1),1) AS avg1, ROUND(AVG(p2),1) AS avg2, ROUND(AVG(p3),1) AS avg3, ROUND(AVG(p4),1) AS avg4, 
            ROUND(AVG(p5),1) AS avg5, ROUND(AVG(p6),1) AS avg6, ROUND(AVG(p7),1) AS avg7, ROUND(AVG(p8),1) AS avg8,
            ROUND(AVG(p9),1) AS avg9, ROUND(AVG(p10),1) AS avg10, ROUND(AVG(p11),1) AS avg11, ROUND(AVG(p12),1) AS avg12, 
            ROUND(AVG(p13),1) AS avg13, ROUND(AVG(p14),1) AS avg14, ROUND(AVG(p15),1) AS avg15, ROUND(AVG(promedio),1) AS promedio_tot
    FROM vista_evaluacion
    LEFT JOIN vista_pers_equ3 USING (correo)
    WHERE equipo <> vista_evaluacion.equipo_expositor
    GROUP BY vista_evaluacion.equipo_expositor
    """)
nota_item_equ.show(30)


+--------------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+------------+
|    equipo_expositor|avg1|avg2|avg3|avg4|avg5|avg6|avg7|avg8|avg9|avg10|avg11|avg12|avg13|avg14|avg15|promedio_tot|
+--------------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+------------+
|Servicios a domic...| 4.5| 4.6| 4.5| 4.3| 4.5| 4.6| 4.1| 4.1| 4.4|  4.3|  4.2|  4.6|  4.5|  4.6|  4.6|         4.4|
|Lo tengo y te lo ...| 4.5| 4.6| 4.3| 4.4| 4.4| 4.7| 4.4| 4.3| 4.5|  4.5|  4.4|  4.6|  4.5|  4.6|  4.6|         4.5|
|Gestión de Reside...| 4.6| 4.4| 4.6| 4.4| 4.5| 4.1| 4.5| 4.7| 4.4|  4.5|  4.3|  4.6|  4.4|  4.5|  4.4|         4.5|
|       Te lo cambio.| 4.4| 4.0| 4.2| 4.0| 4.5| 4.8| 4.1| 4.1| 4.2|  4.3|  3.6|  4.5|  4.6|  4.5|  4.5|         4.3|
|Administración de...| 4.1| 4.4| 4.0| 3.9| 4.2| 4.4| 4.0| 4.4| 4.0|  4.4|  4.1|  4.2|  4.3|  4.2|  4.1|         4.2|
|Deserción de estu...| 4.5| 5.0| 5.0| 5.0| 4.5| 5.0| 5.0| 4.5| 5

In [46]:
nota_item_equ.createOrReplaceTempView('vista_nota_item_equ')
ContextoSql.sql('show tables').show()

+--------+-------------------+-----------+
|database|          tableName|isTemporary|
+--------+-------------------+-----------+
|        |   vista_asistencia|       true|
|        |   vista_evaluacion|       true|
|        |vista_nota_item_equ|       true|
|        |     vista_pers_equ|       true|
|        |    vista_pers_equ2|       true|
|        |    vista_pers_equ3|       true|
|        |  vista_xasistencia|       true|
+--------+-------------------+-----------+



In [47]:
max_avg1 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg1
        FROM vista_nota_item_equ
        WHERE avg1 = (SELECT max(avg1) FROM vista_nota_item_equ)
        """)
max_avg1.show()

max_avg2 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg2
        FROM vista_nota_item_equ
        WHERE avg2 = (SELECT max(avg2) FROM vista_nota_item_equ)
        """)
max_avg2.show()

max_avg3 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg3
        FROM vista_nota_item_equ
        WHERE avg3 = (SELECT max(avg3) FROM vista_nota_item_equ)
        """)
max_avg3.show()

max_avg4 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg4
        FROM vista_nota_item_equ
        WHERE avg4 = (SELECT max(avg4) FROM vista_nota_item_equ)
        """)
max_avg4.show()

max_avg5 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg5
        FROM vista_nota_item_equ
        WHERE avg5 = (SELECT max(avg5) FROM vista_nota_item_equ)
        """)
max_avg5.show()

max_avg6 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg6
        FROM vista_nota_item_equ
        WHERE avg6 = (SELECT max(avg6) FROM vista_nota_item_equ)
        """)
max_avg6.show()

max_avg7 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg7
        FROM vista_nota_item_equ
        WHERE avg7 = (SELECT max(avg7) FROM vista_nota_item_equ)
        """)
max_avg7.show()

max_avg8 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg8
        FROM vista_nota_item_equ
        WHERE avg8 = (SELECT max(avg8) FROM vista_nota_item_equ)
        """)
max_avg8.show()

max_avg9 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg9
        FROM vista_nota_item_equ
        WHERE avg9 = (SELECT max(avg9) FROM vista_nota_item_equ)
        """)
max_avg9.show()

max_avg10 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg10
        FROM vista_nota_item_equ
        WHERE avg10 = (SELECT max(avg10) FROM vista_nota_item_equ)
        """)
max_avg10.show()

max_avg11 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg11
        FROM vista_nota_item_equ
        WHERE avg11 = (SELECT max(avg11) FROM vista_nota_item_equ)
        """)
max_avg11.show()

max_avg12 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg12
        FROM vista_nota_item_equ
        WHERE avg12 = (SELECT max(avg12) FROM vista_nota_item_equ)
        """)
max_avg12.show()

max_avg13 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg13
        FROM vista_nota_item_equ
        WHERE avg13 = (SELECT max(avg13) FROM vista_nota_item_equ)
        """)
max_avg13.show()

max_avg14 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg14
        FROM vista_nota_item_equ
        WHERE avg14 = (SELECT max(avg14) FROM vista_nota_item_equ)
        """)
max_avg14.show()

max_avg15 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg15
        FROM vista_nota_item_equ
        WHERE avg15 = (SELECT max(avg15) FROM vista_nota_item_equ)
        """)
max_avg15.show()

max_promedio_tot = ContextoSql.sql("""
        SELECT  equipo_expositor, promedio_tot
        FROM vista_nota_item_equ
        WHERE promedio_tot = (SELECT max(promedio_tot) FROM vista_nota_item_equ)
        """)
max_promedio_tot.show()

+--------------------+----+
|    equipo_expositor|avg1|
+--------------------+----+
|Gestión de Reside...| 4.6|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg2|
+--------------------+----+
|Deserción de estu...| 5.0|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg3|
+--------------------+----+
|Deserción de estu...| 5.0|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg4|
+--------------------+----+
|Deserción de estu...| 5.0|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg5|
+--------------------+----+
|Servicios a domic...| 4.5|
|Gestión de Reside...| 4.5|
|       Te lo cambio.| 4.5|
|Deserción de estu...| 4.5|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg6|
+--------------------+----+
|Deserción de estu...| 5.0|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg7|
+-------------

In [48]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...
min_avg1 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg1
        FROM vista_nota_item_equ
        WHERE avg1 = (SELECT min(avg1) FROM vista_nota_item_equ)
        """)
min_avg1.show()

min_avg2 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg2
        FROM vista_nota_item_equ
        WHERE avg2 = (SELECT min(avg2) FROM vista_nota_item_equ)
        """)
min_avg2.show()

min_avg3 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg3
        FROM vista_nota_item_equ
        WHERE avg3 = (SELECT min(avg3) FROM vista_nota_item_equ)
        """)
min_avg3.show()

min_avg4 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg4
        FROM vista_nota_item_equ
        WHERE avg4 = (SELECT min(avg4) FROM vista_nota_item_equ)
        """)
min_avg4.show()

min_avg5 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg5
        FROM vista_nota_item_equ
        WHERE avg5 = (SELECT min(avg5) FROM vista_nota_item_equ)
        """)
min_avg5.show()

min_avg6 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg6
        FROM vista_nota_item_equ
        WHERE avg6 = (SELECT min(avg6) FROM vista_nota_item_equ)
        """)
min_avg6.show()

min_avg7 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg7
        FROM vista_nota_item_equ
        WHERE avg7 = (SELECT min(avg7) FROM vista_nota_item_equ)
        """)
min_avg7.show()

min_avg8 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg8
        FROM vista_nota_item_equ
        WHERE avg8 = (SELECT min(avg8) FROM vista_nota_item_equ)
        """)
min_avg8.show()

min_avg9 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg9
        FROM vista_nota_item_equ
        WHERE avg9 = (SELECT min(avg9) FROM vista_nota_item_equ)
        """)
min_avg9.show()

min_avg10 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg10
        FROM vista_nota_item_equ
        WHERE avg10 = (SELECT min(avg10) FROM vista_nota_item_equ)
        """)
min_avg10.show()

min_avg11 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg11
        FROM vista_nota_item_equ
        WHERE avg11 = (SELECT min(avg11) FROM vista_nota_item_equ)
        """)
min_avg11.show()

min_avg12 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg12
        FROM vista_nota_item_equ
        WHERE avg12 = (SELECT min(avg12) FROM vista_nota_item_equ)
        """)
min_avg12.show()

min_avg13 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg13
        FROM vista_nota_item_equ
        WHERE avg13 = (SELECT min(avg13) FROM vista_nota_item_equ)
        """)
min_avg13.show()

min_avg14 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg14
        FROM vista_nota_item_equ
        WHERE avg14 = (SELECT min(avg14) FROM vista_nota_item_equ)
        """)
min_avg14.show()

min_avg15 = ContextoSql.sql("""
        SELECT  equipo_expositor, avg15
        FROM vista_nota_item_equ
        WHERE avg15 = (SELECT min(avg15) FROM vista_nota_item_equ)
        """)
min_avg15.show()

min_promedio_tot = ContextoSql.sql("""
        SELECT  equipo_expositor, promedio_tot
        FROM vista_nota_item_equ
        WHERE promedio_tot = (SELECT min(promedio_tot) FROM vista_nota_item_equ)
        """)
min_promedio_tot.show()

+--------------------+----+
|    equipo_expositor|avg1|
+--------------------+----+
|Administración de...| 4.1|
+--------------------+----+

+----------------+----+
|equipo_expositor|avg2|
+----------------+----+
| Mi Sanandresito| 3.9|
+----------------+----+

+--------------------+----+
|    equipo_expositor|avg3|
+--------------------+----+
|Administración de...| 4.0|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg4|
+--------------------+----+
|Administración de...| 3.9|
|Gestión de progra...| 3.9|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg5|
+--------------------+----+
|Administración de...| 4.2|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg6|
+--------------------+----+
|Gestión de Menús ...| 4.0|
+--------------------+----+

+--------------------+----+
|    equipo_expositor|avg7|
+--------------------+----+
|Administración de...| 4.0|
+--------------------+----+

+----

In [49]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...
nota_estudiante = ContextoSql.sql("""
                SELECT correo, promedio_tot
                FROM vista_pers_equ3
                LEFT JOIN vista_nota_item_equ
                ON vista_nota_item_equ.equipo_expositor = vista_pers_equ3.equipo
                ORDER BY correo
                """)
nota_estudiante.show(20)               


+--------------------+------------+
|              correo|promedio_tot|
+--------------------+------------+
| abanolc@unal.edu.co|         4.4|
|acastrillonv@unal...|         4.4|
| aflemag@unal.edu.co|         4.2|
|anapariciom@unal....|         4.4|
|angutierrezb@unal...|         4.2|
|anoriega@unal.edu.co|         4.3|
|auarbelaeza@unal....|         4.5|
| bocampo@unal.edu.co|         4.4|
|cgiraldo@unal.edu.co|         4.5|
|cjfunezg@unal.edu.co|         4.4|
|cquinchiar@unal.e...|         4.5|
| dadazam@unal.edu.co|         4.2|
|daestradam@unal.e...|         4.5|
|davgarciava@unal....|         4.4|
|dballesteroso@una...|         4.3|
|  dbrito@unal.edu.co|         4.5|
|dcadavid@unal.edu.co|         4.2|
|dcardonaal@unal.e...|         4.5|
|dchavarriar@unal....|         4.4|
|dgarciabl@unal.ed...|         4.5|
+--------------------+------------+
only showing top 20 rows

